# Standard Least Squared
- No preprosessing(not even polynomials), plotting or anything fancy
- Plain Least Squared from exercise 3

###### Next steps: 

Low-fruit:
- make polynomials
- implement cross validation

High fruit:
- Maybe make a new document with only the Exploratory data analysis with PCA and such. Then we different algorithms can choose to highlight some result and use it in their Feature prosessing. (https://en.wikipedia.org/wiki/Exploratory_data_analysis)
- find or write code that finds the best degrees() can't plot...or can just plot one dimension at a time, better to just use ridge regression at that point? Does LS have any type of overfitting add-on

### Libraries and imports

In [48]:
# standard libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# own functions
from proj1_helpers import * 
import basic_functions as bf
#import plot_functions as pf 

#constants
train_path = 'train.csv'
test_path = 'test.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Importing data

In [138]:
y, x, ids = load_csv_data(train_path, sub_sample=True) #remember to switch of subsample when running it "for real"

x = clean_data(x, mean = True)

x

mean:  121.706884347     col:  0
mean:  50.2747695539     col:  1
mean:  81.0094476895     col:  2
mean:  57.2444492899     col:  3
mean:  2.40931254237     col:  4
mean:  366.297965424     col:  5
mean:  -0.795862372881     col:  6
mean:  2.37568113623     col:  7
mean:  18.9787617524     col:  8
mean:  157.932377075     col:  9
mean:  1.44765393079     col:  10
mean:  -0.15876035207     col:  11
mean:  0.475420338983     col:  12
mean:  37.9705389078     col:  13
mean:  -0.00728345669134     col:  14
mean:  0.000356071214243     col:  15
mean:  46.3437475495     col:  16
mean:  -0.0197271454291     col:  17
mean:  0.115021204241     col:  18
mean:  42.2081972394     col:  19
mean:  0.0285643128626     col:  20
mean:  209.392883777     col:  21
mean:  0.984396879376     col:  22
mean:  84.8364731254     col:  23
mean:  -0.0171522893165     col:  24
mean:  -0.00245487723955     col:  25
mean:  57.7075864407     col:  26
mean:  -0.00641288135593     col:  27
mean:  -0.12118440678     co

array([[  1.38470000e+02,   5.16550000e+01,   9.78270000e+01, ...,
          1.24000000e+00,  -2.47500000e+00,   1.13497000e+02],
       [  2.19057000e+02,   7.24610000e+01,   1.24835000e+02, ...,
         -6.41288136e-03,  -1.21184407e-01,   5.03960000e+01],
       [  9.08010000e+01,   2.77870000e+01,   6.53730000e+01, ...,
         -6.41288136e-03,  -1.21184407e-01,   6.27660000e+01],
       ..., 
       [  1.42347000e+02,   7.38900000e+00,   9.92120000e+01, ...,
         -6.41288136e-03,  -1.21184407e-01,   9.70680000e+01],
       [  7.81620000e+01,   4.63350000e+01,   6.01360000e+01, ...,
         -6.41288136e-03,  -1.21184407e-01,   3.24400000e+01],
       [  1.30042000e+02,   4.07300000e+00,   6.78190000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   5.10370000e+01]])

<h3>Cleaning Data</h3>



In [135]:
def clean_data(data, replace_9mean = False):
    
    if(mean):
        
        rows, columns = data.shape
        column_means = np.zeros(30)

        #Manually counting mean for each column without -999 values. 
        for col in range(columns):
            c_count = 0
            c_sum = 0
            for row in range(rows - 1):
                data_point = data[row][col]
                
                if(data_point != -999):
                    c_count += 1
                    c_sum += data_point
            
            mean_mean = c_sum / c_count            
            column_means[col] = mean_mean
        
        #Replacing -999 values with column mean
        for col in range(columns - 1):
            for row in range(rows - 1):
                
                data_point = data[row][col]
                
                if(data_point == -999):
                    data[row][col] = column_means[col]  
                              
    else:
        data[data == -999] = 0
    
    return data

### Split data

In [51]:
seed = 1
ratio = 0.8

x_tr, x_te, y_tr, y_te = bf.split_data(x, y, ratio, seed)
print(x_tr.shape, x_te.shape, y_tr.shape, y_te.shape)

(4000, 30) (1000, 30) (4000,) (1000,)


### Exploratory data analysis (ex: PCA)
https://en.wikipedia.org/wiki/Exploratory_data_analysis

### Feature Processing (ex:Fourier, Poly)
http://machinelearningmastery.com/discover-feature-engineering-how-to- engineer-features-and-how-to-get-good-at-it/.

Ideas:
- standardize
- polynomials
- fourier transformation
- sine, cos, tan ...ect
- look up kvantemechanics formulas an see if we can find some correlations

### Train and produce weigths 

In [52]:
initial_w = np.zeros(len(x[1]))
max_iters = 15 # low
gamma = 0.5
tx = np.transpose(x) 
ty = np.transpose(y)

losses, w = bf.least_squares(ty, x)

print(w.shape)


(30,)


### Optimalizing: determining overfit or underfit
http://cs229.stanford.edu/materials/ML-advice.pdf

### Visualize, find pattern of wrong predictions -> back to Exploratory/Feature Proc
http://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

### Cross validation before kaggle

### Predict higgs boson

In [53]:
# Import test.csv which contains nonlabeled data
# Remember to have sub_sample=False when submitting to kaggle WARNING: it takes a looooong time
pred_y, pred_x, pred_ids = load_csv_data(test_path, sub_sample=False)
print(pred_x, pred_y)
print(pred_ids)

[[-999.      79.589   23.916 ..., -999.    -999.       0.   ]
 [ 106.398   67.49    87.949 ..., -999.    -999.      47.575]
 [ 117.794   56.226   96.358 ..., -999.    -999.       0.   ]
 ..., 
 [ 108.497    9.837   65.149 ..., -999.    -999.       0.   ]
 [  96.711   20.006   66.942 ..., -999.    -999.      30.863]
 [  92.373   80.109   77.619 ..., -999.    -999.       0.   ]] [ 1.  1.  1. ...,  1.  1.  1.]
[350000 350001 350002 ..., 918235 918236 918237]


In [55]:
weights = w

#predictions = compute_prediction(pred_x, weights)
#print(predictions[1:25])

y_predictions = predict_labels(weights, pred_x)
print(y_predictions[1:25])


[-1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.  1.]


In [56]:
# creates a file with the name you want in the folder of the Python-file. just look in the folder :) 
name = 'powerpuff_testing.csv'
create_csv_submission(pred_ids, y_predictions, name)

print("yolo")

#
# THE END
#

yolo
